In [3]:
import mindspore
import mindspore.nn as nn
import mindspore.numpy as mnp

In [2]:
MAX_VOCAB_SIZE = 25000

In [4]:
class RNN(nn.Cell):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers,
                 bidirectional, dropout, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        self.rnn = nn.LSTM(embedding_dim,
                           hidden_dim,
                           num_layers=n_layers,
                           bidirectional=bidirectional,
                           dropout=dropout,
                           batch_first=True)
        self.fc = nn.Dense(hidden_dim * 2, output_dim)
        self.dropout = nn.Dropout(1 - dropout)
    
    def construct(self, inputs, seq_lengths):
        # inputs: (batch, seq_length)
        embedded = self.dropout(self.embedding(inputs))
        # embedded: (batch, seq_length, embedding_dim)
        outputs, (hidden, cell) = self.rnn(embedded, seq_length=seq_lengths)
        #outputs: (batch, seq_length, hidden_dim * num_directions)
        #hidden: (num_layers * num_directions, batch_size, hidden_dim)
        #cell: (num_layers * num_directions, batch_size, hidden_dim)
        hidden = self.dropout(mnp.concatenate((hidden[-2,:,:], hidden[-1,:,:]), axis = 1))
        return self.fc(hidden)

In [6]:
INPUT_DIM = 200
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = 0

model = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX)